In [1]:
import numpy as np
import pandas as pd
from penquins import Kowalski
from scope.utils import write_parquet

# Authenticate kowalski, gloria, melman (first time)

In [ ]:
kowalski = Kowalski(username = '',
                    password = '',
                    host = 'kowalski.caltech.edu',
                   )


gloria = Kowalski(username = '',
                  password = '',
                  host = 'gloria.caltech.edu',
                 )

melman = Kowalski(username = '',
                  password = '',
                  host = 'melman.caltech.edu',
                 )

In [ ]:
# Generate tokens (no need to do this every time)
k_token = kowalski.authenticate()
g_token = gloria.authenticate()
m_token = melman.authenticate()

# Print these tokens to add them to your config.yaml file

In [2]:
# Fill in tokens here after first authentication
tokens = {'kowalski': '', #k_token
          'gloria': '', #g_token
          'melman': '', #m_token
         }

# Create kowalski, gloria, melman instances using token

In [3]:
hosts = ['kowalski', 'gloria', 'melman']

instances = {
    host: {
        'protocol': 'https',
        'port': 443,
        'host': f'{host}.caltech.edu',
        'token': tokens[host],
    }
    for host in hosts
}

kowalski_instances = Kowalski(instances=instances, timeout=610)

In [4]:
# Test connection
for host in hosts:
    print(host, kowalski_instances.ping(name=host))

kowalski True
gloria True
melman True


# Get kowalski, gloria, melman catalog names

In [5]:
query = {
    "query_type": "info",
    "query": {
        "command": "catalog_names",
    }
}

# For queries without a catalog, specify the instance name

response_k = kowalski_instances.query(query=query, name='kowalski')
data_k = response_k.get("kowalski").get("data")

response_g = kowalski_instances.query(query=query, name='gloria')
data_g = response_g.get("gloria").get("data")

response_m = kowalski_instances.query(query=query, name='melman')
data_m = response_m.get("melman").get("data")

print(response_m)

{'melman': {'status': 'success', 'message': 'Successfully executed query', 'data': ['ZTF_sources_84525009', 'ZTF_sources_60187478', 'ZTF_sources_32604668', 'ZTF_sources_26553423', 'ZTF_sources_20230309', 'ZTF_source_features_DR16', 'ZTF_source_features_20201201', 'ZTF_source_classifications_DR5', 'ZTF_source_classifications_DR16', 'ZTF_public_sources_20210401', 'ZTF_ops', 'ZTF_exposures_84525009', 'ZTF_exposures_60187478', 'ZTF_exposures_32604668', 'ZTF_exposures_26553423', 'ZTF_exposures_20230309', 'ZTF_alerts', 'WNTR_alerts', 'VLASS_DR1', 'TURBO_alerts', 'TNS', 'PTF_sources', 'PTF_exposures', 'PS1_DR1', 'PGIR_alerts', 'IGAPS_DR2', 'Gaia_EDR3', 'GALEX', 'AllWISE']}}


In [6]:
data_k, data_g, data_m

(['skymaps',
  'sdss_ellipticals',
  'mzls_ellipticals',
  'milliquas_v6',
  'legacysurveys_photoz_DR7',
  'legacysurveys_photoz_DR6',
  'galaxy_redshifts_20200522',
  'cfht_w3_photozs',
  'ZUDS_alerts_aux',
  'ZUDS_alerts',
  'ZTF_sources_20200401',
  'ZTF_source_features_20201201',
  'ZTF_source_features_20200401_20_fields',
  'ZTF_source_features_20191101_20_fields',
  'ZTF_source_features_20191101',
  'ZTF_source_classifications_dr2_20_fields',
  'ZTF_source_classifications_dr2',
  'ZTF_source_classifications_20191101',
  'ZTF_ops',
  'ZTF_exposures_20201201',
  'ZTF_exposures_20200401',
  'ZTF_exposures_20191101',
  'ZTF_alerts_aux',
  'ZTF_alerts',
  'VLASS_DR1',
  'TNS',
  'TGSS_ADR1',
  'RFC_2019d',
  'RFC_2019a',
  'PS1_STRM',
  'PS1_DR1',
  'PGIR_alerts_aux',
  'PGIR_alerts',
  'NVSS_41',
  'NED_BetaV3',
  'LAMOST_DR5_v3',
  'LAMOST_DR4_v2',
  'Known_lenses_20180901',
  'IPHAS_DR2',
  'IGAPS_DR2',
  'Gaia_EDR3',
  'Gaia_DR2_light_curves',
  'Gaia_DR2_WD',
  'Gaia_DR2_2MASS_be

# Query instances for ZTF high-confidence IDs

In [ ]:
# For queries with a catalog, no need to specify instance name

qry = kowalski_instances.query({
            "query_type": "find", # find documents
            "query": {
            "catalog": "ZTF_source_classifications_DR16", # catalog name
            "filter": {
                "$or":[
                    {'%s_xgb' % 'vnv': {'$gt': 0.99}}, # filter by vnv_xgb > 0.9 or vnv_dnn > 0.9
                                {'%s_dnn' % 'vnv': {'$gt': 0.99}},
                ]}
            },
            "kwargs": {
                       "max_time_ms": 600000 # adjustable max time in milliseconds
            },
            "projection": {"_id": 1 # only return the ids
                          }
            })
#data = qry.get('data')

#### Query runs too long and can time out - we have to query in batches 

# Query instances for ZTF DR5 classifications

## Get estimated document count

In [7]:
qry = kowalski_instances.query({
    "query_type": "estimated_document_count",
    "query": {
    "catalog": "ZTF_source_classifications_DR16"
        }
    }
)

estimated_nlightcurves = qry.get('melman').get('data')
estimated_nlightcurves

58778905

In [8]:
qry = kowalski_instances.query({
    "query_type": "estimated_document_count",
    "query": {
    "catalog": "ZTF_source_classifications_DR16"
        }
    }
)

for instance_name in qry:
    result = qry.get(instance_name).get('data')
    print(instance_name, result)

result

melman 58778905


58778905

## Get collection info

In [9]:
qry = kowalski_instances.query({
    "query_type": "info",
    "query": {
    "catalog": "ZTF_source_classifications_DR16",
        "command": "catalog_info"
        }
    }
)

## Count number of high-vnv-confidence documents

In [10]:
qry = kowalski_instances.query({
    "query_type": "count_documents",
    "query": {
    "catalog": "ZTF_source_classifications_DR16",
    "kwargs": {
                "max_time_ms": 600000
            },
    "filter": {
        "$or":[
                {'%s_xgb' % 'vnv': {'$gt': 0.99}},
                {'%s_dnn' % 'vnv': {'$gt': 0.99}},
        ]}
    }
})

nlightcurves = qry.get('melman').get('data')
nlightcurves

643340

## Note ability to loop over all instances without specifically naming them

In [11]:
qry = kowalski_instances.query({
    "query_type": "count_documents",
    "query": {
    "catalog": "ZTF_source_classifications_DR16",
    "filter": {
        "$or":[
                {'%s_xgb' % 'vnv': {'$gt': 0.99}},
                {'%s_dnn' % 'vnv': {'$gt': 0.99}},
        ]}
    }
})

# If ZTF_source_classifications_DR5 were on multiple instances, queries could be split between them
nlightcurves = 0

# Loop over instances, updating quantity of interest
for instance_name in qry:
    result = qry.get(instance_name).get('data')
    nlightcurves += result
    print(instance_name, result)

nlightcurves

melman 643340


643340

# Get scores using batch query

In [12]:
ids = []
batch_size=10000
nb = np.ceil(nlightcurves/batch_size).astype(int)
df_all = []

print(f'{nb}:')
for n in range(nb):
    print(n)
    qry = kowalski_instances.query({
        "query_type": "find",
        "query": {
            "catalog": "ZTF_source_classifications_DR16",
            "filter": {
                "$or":[
                    {'%s_xgb' % 'vnv': {'$gt': 0.99}},
                    {'%s_dnn' % 'vnv': {'$gt': 0.99}},
                ]}
            },
        "kwargs": {"skip": int(n*batch_size),
                   "limit": int(batch_size),
                   "max_time_ms": 600000
            }
        })
    data = qry.get('melman').get('data')
    df_tmp = pd.DataFrame.from_records(data)
    df_all += [df_tmp]
    
df_scores = pd.concat(df_all).reset_index(drop=True)
    

65:
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64


In [13]:
# examine last entry of data
data[-1]

{'_id': 10488352164728,
 'Gaia_EDR3___id': 4292955537630067072,
 'AllWISE___id': 0,
 'PS1_DR1___id': 114232913156860688,
 'ra': 291.3158181,
 'dec': 5.1918421,
 'period': 402.2276744186922,
 'field': 488,
 'ccd': 9,
 'quad': 4,
 'filter': 2,
 'e_dnn': 0.0,
 'cv_dnn': 0.0,
 'el_dnn': 0.0,
 'osarg_dnn': 0.0,
 'ea_dnn': 0.75,
 'rrblz_dnn': 0.0,
 'ceph2_dnn': 0.0,
 'bis_dnn': 0.0,
 'puls_dnn': 0.009999999776482582,
 'agn_dnn': 0.0,
 'longt_dnn': 0.0,
 'mir_dnn': 0.0,
 'dscu_dnn': 0.0,
 'rrc_dnn': 0.0,
 'eb_dnn': 0.0,
 'wp_dnn': 0.0,
 'rrd_dnn': 0.0,
 'hp_dnn': 0.009999999776482582,
 'blend_dnn': 0.0,
 'wvir_dnn': 0.0,
 'lpv_dnn': 0.0,
 'emsms_dnn': 0.0,
 'i_dnn': 0.0,
 'rrlyr_dnn': 0.0,
 'rrab_dnn': 0.0,
 'sin_dnn': 0.0,
 'rscvn_dnn': 0.0,
 'bright_dnn': 0.0,
 'bogus_dnn': 0.0,
 'yso_dnn': 0.0,
 'blyr_dnn': 0.0,
 'dp_dnn': 0.0,
 'saw_dnn': 0.17000000178813934,
 'ceph_dnn': 0.0,
 'blher_dnn': 0.0,
 'ext_dnn': 0.0,
 'wuma_dnn': 0.0,
 'ew_dnn': 0.0,
 'srv_dnn': 0.0,
 'fla_dnn': 0.019999999552

In [14]:
df_scores

,_id,Gaia_EDR3___id,AllWISE___id,PS1_DR1___id,ra,dec,period,field,ccd,quad,...,hp_xgb,blyr_xgb,rscvn_xgb,rrd_xgb,mir_xgb,rrc_xgb,bogus_xgb,rrab_xgb,agn_xgb,coordinates
0,10487361000126,4285763524683958528,0,115642824603100448,282.460330,6.366678,0.432570,487,10,1,...,0.01,0.00,0.00,0.0,0.0,0.0,0.02,0.40,0.00,"{'radec_str': ['18:49:50.4792', '06:22:00.040'..."
1,10487361000870,4285433881660412800,0,115562821782080352,282.178217,6.299935,1.260361,487,10,1,...,0.00,0.44,0.00,0.0,0.0,0.0,0.00,0.19,0.00,"{'radec_str': ['18:48:42.7720', '06:17:59.766'..."
2,10487361001120,4285433709861720448,2825106001351041536,115532821501752224,282.150171,6.276463,110.444123,487,10,1,...,0.01,0.00,0.00,0.0,0.0,0.0,0.05,0.33,0.00,"{'radec_str': ['18:48:36.0410', '06:16:35.268'..."
3,10487361002857,4285380486626796160,0,115362824160645344,282.416051,6.137439,1.538381,487,10,1,...,0.00,0.01,0.00,0.0,0.0,0.0,0.00,0.22,0.00,"{'radec_str': ['18:49:39.8522', '06:08:14.780'..."
4,10487361003026,4285740164362451328,2825106001351030784,115352826534580096,282.653441,6.124625,0.021669,487,10,1,...,0.01,0.01,0.03,0.0,0.0,0.0,0.01,0.20,0.00,"{'radec_str': ['18:50:36.8257', '06:07:28.652'..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
643335,10488352164538,0,0,113772909831912880,290.983166,4.810384,0.047171,488,9,4,...,0.00,0.00,0.00,0.0,0.0,0.0,0.00,0.19,0.01,"{'radec_str': ['19:23:55.9599', '04:48:37.383'..."
643336,10488352164566,4292715741020464768,0,113692917302599904,291.730300,4.749587,0.104595,488,9,4,...,0.00,0.00,0.00,0.0,0.0,0.0,0.01,0.20,0.00,"{'radec_str': ['19:26:55.2720', '04:44:58.514'..."
643337,10488352164621,4294468328175322752,0,114582917148525424,291.714840,5.487507,0.176596,488,9,4,...,0.01,0.00,0.00,0.0,0.0,0.0,0.00,0.19,0.00,"{'radec_str': ['19:26:51.5616', '05:29:15.024'..."
643338,10488352164635,4292978867891376128,0,114552914554329536,291.455392,5.466413,0.024923,488,9,4,...,0.00,0.00,0.00,0.0,0.0,0.0,0.01,0.19,0.00,"{'radec_str': ['19:25:49.2942', '05:27:59.088'..."


In [15]:
df_scores.columns

Index(['_id', 'Gaia_EDR3___id', 'AllWISE___id', 'PS1_DR1___id', 'ra', 'dec',
       'period', 'field', 'ccd', 'quad', 'filter', 'e_dnn', 'cv_dnn', 'el_dnn',
       'osarg_dnn', 'ea_dnn', 'rrblz_dnn', 'ceph2_dnn', 'bis_dnn', 'puls_dnn',
       'agn_dnn', 'longt_dnn', 'mir_dnn', 'dscu_dnn', 'rrc_dnn', 'eb_dnn',
       'wp_dnn', 'rrd_dnn', 'hp_dnn', 'blend_dnn', 'wvir_dnn', 'lpv_dnn',
       'emsms_dnn', 'i_dnn', 'rrlyr_dnn', 'rrab_dnn', 'sin_dnn', 'rscvn_dnn',
       'bright_dnn', 'bogus_dnn', 'yso_dnn', 'blyr_dnn', 'dp_dnn', 'saw_dnn',
       'ceph_dnn', 'blher_dnn', 'ext_dnn', 'wuma_dnn', 'ew_dnn', 'srv_dnn',
       'fla_dnn', 'vnv_dnn', 'pnp_dnn', 'dip_dnn', 'mp_dnn', 'saw_xgb',
       'dp_xgb', 'i_xgb', 'longt_xgb', 'yso_xgb', 'wvir_xgb', 'puls_xgb',
       'sin_xgb', 'dscu_xgb', 'blend_xgb', 'dip_xgb', 'mp_xgb', 'pnp_xgb',
       'vnv_xgb', 'rrblz_xgb', 'ew_xgb', 'osarg_xgb', 'fla_xgb', 'srv_xgb',
       'ext_xgb', 'ceph2_xgb', 'bis_xgb', 'blher_xgb', 'e_xgb', 'ea_xgb',
       'wuma

In [ ]:
df_scores.to_csv('ZTF_DR16_vnv_gt0.99score_classifications.csv',index=False)

# Batch query to get features

In [16]:
source_ids = df_scores['_id'].values.tolist()
features_catalog = 'ZTF_source_features_DR16'
limit=1000

In [17]:
# Only get the first 100000 feature rows
# Skip this cell to get all rows of features (takes longer/more memory)
source_ids = source_ids[:100000]

In [30]:
# Get features and dmdt from melman
id = 0
df_collection = []
dmdt_collection = []

while 1:
    query = {
        "query_type": "find",
        "query": {
            "catalog": features_catalog,
            "filter": {"_id": {"$in": source_ids[id * limit : (id + 1) * limit]}},
        },
    }
    response = kowalski_instances.query(query=query)
    source_data = response.get('melman').get("data")

    if source_data is None:
        print(response)
        raise ValueError(f"No data found for source ids {source_ids}")

    df_temp = pd.DataFrame.from_records(source_data)
    df_collection += [df_temp]
    try:
        dmdt_temp = np.expand_dims(
            np.array([d for d in df_temp['dmdt'].values]), axis=-1
        )
    except Exception as e:
        print("Error", e)
        print(df_temp)
    dmdt_collection += [dmdt_temp]

    if ((id + 1) * limit) >= len(source_ids):
        break
    id += 1
    if (id * limit) % 5000 == 0:
        print(id * limit, "done")

df_features = pd.concat(df_collection, axis=0)
dmdt = np.vstack(dmdt_collection)

5000 done
10000 done
15000 done
20000 done
25000 done
30000 done
35000 done
40000 done
45000 done
50000 done
55000 done
60000 done
65000 done
70000 done
75000 done
80000 done
85000 done
90000 done
95000 done


In [31]:
df_features

,_id,ra,dec,field,ccd,quad,filter,avg_err,med_err,std_err,...,PS1_DR1__rMeanPSFMag,PS1_DR1__rMeanPSFMagErr,PS1_DR1__iMeanPSFMag,PS1_DR1__iMeanPSFMagErr,PS1_DR1__zMeanPSFMag,PS1_DR1__zMeanPSFMagErr,PS1_DR1__yMeanPSFMag,PS1_DR1__yMeanPSFMagErr,PS1_DR1__qualityFlag,coordinates
0,10487361000126,282.460330,6.366678,487,10,1,1,0.158327,0.161550,0.029707,...,18.619301,0.020096,17.481199,0.003968,16.4561,0.030932,15.871600,0.017195,60.0,"{'radec_str': ['18:49:50.4792', '06:22:00.040'..."
1,10487361000870,282.178217,6.299935,487,10,1,1,0.043037,0.042940,0.000721,...,15.714000,0.008957,14.807700,0.001568,14.2851,0.000326,13.901300,0.003348,60.0,"{'radec_str': ['18:48:42.7720', '06:17:59.766'..."
2,10487361001120,282.150171,6.276463,487,10,1,1,0.179086,0.179845,0.023020,...,17.511101,0.013439,13.948000,0.000113,11.9440,NaN,11.076000,NaN,60.0,"{'radec_str': ['18:48:36.0410', '06:16:35.268'..."
3,10487361002857,282.416051,6.137439,487,10,1,1,0.041611,0.041530,0.000646,...,15.746600,0.003946,14.786300,0.007354,14.3287,0.028565,13.959100,0.013341,60.0,"{'radec_str': ['18:49:39.8522', '06:08:14.780'..."
4,10487361003026,282.653441,6.124625,487,10,1,1,0.205640,0.204465,0.017754,...,18.011000,0.038667,13.430000,NaN,11.8470,NaN,11.121000,NaN,60.0,"{'radec_str': ['18:50:36.8257', '06:07:28.652'..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,10778511009049,66.466876,56.988133,778,13,4,1,0.041406,0.040070,0.004709,...,17.784901,0.017927,17.452299,0.009622,17.2747,0.004012,17.100300,0.007755,60.0,"{'radec_str': ['04:25:52.0502', '56:59:17.278'..."
996,10778511009242,66.733928,56.959832,778,13,4,1,0.205202,0.206245,0.020789,...,20.616301,0.033496,20.195700,0.033678,20.0425,0.023722,19.761999,0.079766,52.0,"{'radec_str': ['04:26:56.1427', '56:57:35.397'..."
997,10778511009481,66.666984,56.946181,778,13,4,1,0.033442,0.032630,0.002931,...,17.180700,0.004842,16.739100,0.007846,16.7134,0.030031,16.459700,0.042119,60.0,"{'radec_str': ['04:26:40.0762', '56:56:46.251'..."
998,10778511009575,66.638130,56.940712,778,13,4,1,0.059245,0.057330,0.007718,...,18.131100,0.033658,17.708500,0.041705,17.5884,0.072244,17.246700,0.009060,60.0,"{'radec_str': ['04:26:33.1512', '56:56:26.565'..."


In [32]:
df_features.columns

Index(['_id', 'ra', 'dec', 'field', 'ccd', 'quad', 'filter', 'avg_err',
       'med_err', 'std_err',
       ...
       'PS1_DR1__rMeanPSFMag', 'PS1_DR1__rMeanPSFMagErr',
       'PS1_DR1__iMeanPSFMag', 'PS1_DR1__iMeanPSFMagErr',
       'PS1_DR1__zMeanPSFMag', 'PS1_DR1__zMeanPSFMagErr',
       'PS1_DR1__yMeanPSFMag', 'PS1_DR1__yMeanPSFMagErr',
       'PS1_DR1__qualityFlag', 'coordinates'],
      dtype='object', length=193)

In [33]:
# Merge features, labels
df_merge = pd.merge(df_scores, df_features, on='_id')

In [ ]:
df_merge.to_csv('golden_merged_scores_features.csv',index=False)

In [ ]:
write_parquet(df_merge, 'golden_merged_scores_features.parquet')